In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [ ]:
%run -p ../train_cnf_disentangle_rl_semisup.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/infocnf_cond_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_6_nsup4k_beta_1000_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 6.0 --labeled_batch_size 450 --num_train_sup 4000 --beta 1000.0
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_semisup.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWr

Load semi-sup datasets for CIFAR10


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0001 | Time 112.6362, Epoch Time 1443.4331(1443.4331), Bit/dim 7.0173(best: inf), Bit/dim Marginal 7.0170(best: inf), Xent 2.0832, Loss 8.0589, Error 0.7076(best: inf)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0110 | Time 13.7558(13.8841) | Bit/dim 6.9789(7.3713) | Xent 2.0808(2.0931) | Loss 18.1608(19.3644) | Error 0.6956(0.7078) Steps 568(525.99) | Grad Norm 8.7333(6.0047) | Total Time 0.00(0.00)
Iter 0120 | Time 13.1449(13.8518) | Bit/dim 6.9533(7.2658) | Xent 2.0297(2.0841) | Loss 17.7937(18.9710) | Error 0.6867(0.7049) Steps 532(529.23) | Grad Norm 3.5183(6.0684) | Total Time 0.00(0.00)
Iter 0130 | Time 13.7124(13.8454) | Bit/dim 6.9220(7.1794) | Xent 2.0018(2.0728) | Loss 17.7757(18.6700) | Error 0.6711(0.7004) Steps 556(530.64) | Grad Norm 4.4384(6.6063) | Total Time 0.00(0.00)
Iter 0140 | Time 14.0008(13.8606) | Bit/dim 6.8954(7.1065) | Xent 1.9783(2.0604) | Loss 17.7023(18.4334) | Error 0.6378(0.6931) Steps 568(535.95) | Grad Norm 4.1636(6.7684) | Total

...

Epoch 0008 | Time 84.0079, Epoch Time 1699.5381(1472.1870), Bit/dim 4.5634(best: 4.6509), Bit/dim Marginal 4.5216(best: 4.6039), Xent 1.7639, Loss 5.4453, Error 0.6180(best: 0.6586)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0820 | Time 15.0094(15.1210) | Bit/dim 4.4834(4.5504) | Xent 1.3547(1.3910) | Loss 12.1104(12.9032) | Error 0.4667(0.4900) Steps 592(594.54) | Grad Norm 25.2815(22.4873) | Total Time 0.00(0.00)
Iter 0830 | Time 15.1973(15.1187) | Bit/dim 4.5023(4.5375) | Xent 1.3936(1.3875) | Loss 12.0795(12.7005) | Error 0.4733(0.4872) Steps 622(596.37) | Grad Norm 26.6487(22.8464) | Total Time 0.00(0.00)
Iter 0840 | Time 15.1117(15.1498) | Bit/dim 4.5158(4.5261) | Xent 1.3991(1.3802) | Loss 12.0700(12.5281) | Error 0.4956(0.4856) Steps 616(601.02) | Grad Norm 22.8175(22.1435) | Total Time 0.00(0.00)
Iter 0850 | Time 15.8589(15.1684) | Bit/dim 4.4632(4.5120) | Xent 1.3604(1.3728) | Loss 12.0831(12.3851) | Error 0.4667(0.4828) Steps 604(601.82) | Grad Norm 31.0626(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0009 | Time 96.5742, Epoch Time 1686.8458(1478.6268), Bit/dim 4.4611(best: 4.5634), Bit/dim Marginal 4.4266(best: 4.5216), Xent 1.7940, Loss 5.3581, Error 0.6412(best: 0.6180)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0930 | Time 14.6406(15.0530) | Bit/dim 4.4298(4.4333) | Xent 1.2854(1.3479) | Loss 11.8524(12.4525) | Error 0.4444(0.4735) Steps 598(599.94) | Grad Norm 14.9556(22.7463) | Total Time 0.00(0.00)
Iter 0940 | Time 14.7519(15.0939) | Bit/dim 4.3744(4.4199) | Xent 1.2074(1.3322) | Loss 11.7982(12.2681) | Error 0.4178(0.4691) Steps 592(599.84) | Grad Norm 13.3621(20.0963) | Total Time 0.00(0.00)
Iter 0950 | Time 14.8434(15.0617) | Bit/dim 4.3709(4.4048) | Xent 1.1440(1.3144) | Loss 11.6152(12.1044) | Error 0.4378(0.4651) Steps 586(597.46) | Grad Norm 11.5118(17.9635) | Total Time 0.00(0.00)
Iter 0960 | Time 16.0128(15.0641) | Bit/dim 4.6142(4.4013) | Xent 1.4269(1.3116) | Loss 12.3409(12.0181) | Error 0.4978(0.4631) Steps 652(598.53) | Grad Norm 33.0322(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0010 | Time 84.7088, Epoch Time 1674.5948(1484.5058), Bit/dim 4.3037(best: 4.4611), Bit/dim Marginal 4.2791(best: 4.4266), Xent 1.7054, Loss 5.1564, Error 0.6019(best: 0.6180)
===> Using batch size 900. Total 102 iterations/epoch.
